<a href="https://colab.research.google.com/github/mingrui-liu/Hyperpigmentation-autograde/blob/main/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import shutil
import random
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np

import tensorflow_addons as tfa
from tensorflow.keras.optimizers import SGD,Adam,RMSprop,Nadam

In [ ]:
# connect to your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64
IMG_SIZE = 224

In [ ]:
#Create image tf dataset from directory 
AUTOTUNE = tf.data.experimental.AUTOTUNE

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  return image

all_image_paths_1 = sorted(glob.glob("/content/drive/My Drive/Unilever/image_test/*.jpg"))
all_image_paths_2 = sorted(glob.glob("/content/drive/My Drive/Unilever/Additional_Images/*.jpg"))

# read the excel file
MHPgrades = pd.read_excel("/content/drive/My Drive/Unilever/MHPgrades.xlsx",header = 0)
MHPgrades2 = pd.read_excel("/content/drive/My Drive/Unilever/SKN-APP-0452-MHPgrades.xlsx",header = 1)

MHPgrades2 = MHPgrades2.dropna(axis = 0)
MHPgrades2 = MHPgrades2.melt(id_vars=["Subject No.", "Side of Face"], 
        var_name="VISIT", 
        value_name="Mottled hyperpigmentation").sort_values(by = ['Subject No.','Side of Face'])

grade1 = MHPgrades["Mottled hyperpigmentation"]

#Manipulate excel file
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Baseline' , 'VISIT'] = '00'
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Week 4' , 'VISIT'] = '04'
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Week 8' , 'VISIT'] = '08'
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Week 12' , 'VISIT'] = '12'
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Week 16' , 'VISIT'] = '16'
MHPgrades2.loc[MHPgrades2['VISIT'] == 'Week 17' , 'VISIT'] = '17'

MHPgrades2.loc[MHPgrades2['Side of Face'] == 'Left' , 'Side of Face'] = 'L'
MHPgrades2.loc[MHPgrades2['Side of Face'] == 'Right' , 'Side of Face'] = 'R'

#match grade and images and make valid pair
valid_path = []
valid_label = []
for index, row in MHPgrades2.iterrows():
    id = str(row['Subject No.'] )
    id = id.zfill(4)

    side = row['Side of Face']
   
    visit = str(row['VISIT'])

    grade = row['Mottled hyperpigmentation']
  


    output = [path for path in all_image_paths_2 if path.startswith('/content/drive/My Drive/Unilever/Additional_Images/P' + id + '_FACE_' + side + '_S00_VCR_S2_V00D00W' + visit)] 
    if len(output)>0:
      valid_path.append(output[0])
      valid_label.append(grade)

all_image_paths = all_image_paths_1 + valid_path
all_image_labels = list(grade1) +  valid_label
print(len(all_image_labels ))
print(len(all_image_paths))


In [ ]:
##Create label and zip dataset
all_label = []
for grade in all_image_labels:
  if grade < 2.0:
    label = 1.5
  elif grade > 4.0:
    label = 4.5
  else:
    label = grade
  all_label.append(label)

label_to_index = dict((name, index) for index, name in enumerate(sorted(set(all_label))))
all_labels = [label_to_index[grade] for grade in all_label]

from sklearn.model_selection import train_test_split
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels) 

path_ds = tf.data.Dataset.from_tensor_slices(train_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
DATASET_SIZE = len(image_ds)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(train_labels, tf.float32))

# a dataset that returns images and labels
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

using generator

In [ ]:

# here's our final training dataset
train_ds = image_label_ds.cache()
train_ds = train_ds.shuffle(len(train_ds))
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# let's make a test dataset as well
path_ds = tf.data.Dataset.from_tensor_slices(test_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(test_labels, tf.float32))
image_label_ds_test = tf.data.Dataset.zip((image_ds, label_ds))
test_ds = image_label_ds_test.cache().batch(BATCH_SIZE)

### base model


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', 
                        input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(7, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=5)

#### Transfer Learning

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(include_top = False, weights = 'imagenet', 
                         input_shape = (IMG_SIZE, IMG_SIZE, 3))



for layer in base_model.layers[:20]:
  layer.trainable = False

drop_out = tf.keras.layers.Dropout(.2, input_shape =(24, 16, 16,2048))
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')

# build a new model reusing the pretrained base
model = tf.keras.Sequential([
  base_model,
  drop_out,
  global_average_layer,
  prediction_layer
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

#  no rescale  no augmentation ,trainable = true
history = model.fit(train_ds,
      epochs=50,
      callbacks=[callback],
      validation_data = test_ds)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
def plot(history):
  
    plt.title('Training and validation accuracy')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show()


    plt.figure()
    plt.title('Training and validation loss')
    plt.plot(history.history['loss'][7:], label='loss')
    plt.plot(history.history['val_loss'][7:], label = 'val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# build a new model reusing the pretrained base
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(include_top = False, weights = 'imagenet', 
                         input_shape = (IMG_SIZE, IMG_SIZE, 3))


for layer in base_model.layers[:20]:
  layer.trainable = False

drop_out = tf.keras.layers.Dropout(.4, input_shape =(24, 16, 16,2048))
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')


model2 = tf.keras.Sequential([
  base_model,
  drop_out,
  global_average_layer,
  prediction_layer
])


opt = tf.keras.optimizers.Adam(learning_rate=1e-6)

model2.compile(optimizer= opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history2 = model2.fit(train_ds,
      epochs = 100,
      validation_data = test_ds)

Epoch 1/50
29/29 [==============================] - 18s 634ms/step - loss: 2.4760 - accuracy: 0.1171 - val_loss: 2.8499 - val_accuracy: 0.1398
Epoch 2/50
29/29 [==============================] - 18s 613ms/step - loss: 2.3190 - accuracy: 0.1199 - val_loss: 2.6546 - val_accuracy: 0.1398
Epoch 3/50
29/29 [==============================] - 17s 600ms/step - loss: 2.1883 - accuracy: 0.1310 - val_loss: 2.4665 - val_accuracy: 0.1464
Epoch 4/50
29/29 [==============================] - 17s 594ms/step - loss: 2.0711 - accuracy: 0.1559 - val_loss: 2.2877 - val_accuracy: 0.1564
Epoch 5/50
29/29 [==============================] - 17s 596ms/step - loss: 1.9695 - accuracy: 0.1881 - val_loss: 2.1287 - val_accuracy: 0.1697
Epoch 6/50
29/29 [==============================] - 17s 599ms/step - loss: 1.8767 - accuracy: 0.2175 - val_loss: 2.0100 - val_accuracy: 0.1797
Epoch 7/50
29/29 [==============================] - 17s 602ms/step - loss: 1.7880 - accuracy: 0.2664 - val_loss: 1.9150 - val_accuracy: 0.2196

In [ ]:
plot(history2)